In [4]:
# test
c = -0.61661-0.42333j
z = 0+0j
for n in range(9):
    z = z*z + c
    print("{}: z={:33}, abs(z)={:0.2f},c={}".format(n, z, abs(z), c))

0: z=              (-0.61661-0.42333j), abs(z)=0.75,c=(-0.61661-0.42333j)
1: z=(-0.4156103968+0.09872902259999994j), abs(z)=0.43,c=(-0.61661-0.42333j)
2: z=(-0.4536254179752979-0.5053956165169242j), abs(z)=0.68,c=(-0.61661-0.42333j)
3: z=(-0.6662587093612582+0.03519059557074622j), abs(z)=0.67,c=(-0.61661-0.42333j)
4: z=(-0.17394771021689426-0.47022208157323875j), abs(z)=0.50,c=(-0.61661-0.42333j)
5: z=(-0.8074610001093689-0.25974189123382685j), abs(z)=0.85,c=(-0.61661-0.42333j)
6: z=(-0.032082583364102746-0.003867105468070453j), abs(z)=0.03,c=(-0.61661-0.42333j)
7: z=(-0.6155956623493866-0.4230818665328857j), abs(z)=0.75,c=(-0.61661-0.42333j)
8: z=(-0.41665024628557057+0.09756472371265312j), abs(z)=0.43,c=(-0.61661-0.42333j)


### 2.6 使用cprofile模块分析代码性能

In [ ]:
%python -m cProfile -s cumulative julial_nopil.py

In [1]:
from functools import wraps

In [ ]:
"""
定义一个修饰器来自动测量时间
"""
def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print("@timefn:"+fn.func_name+" took "+str(t2-t1)+"secondes")
        return result
    return measure_time        

In [9]:
# 2.12 dist模块检查cpython字节码
def fn_expression(upper = 100000):
    total = 0 
    for n in range(upper):
        total+=n
    return total

def fn_terse(upper = 100000):
    return sum(range(upper))

print(fn_expression() == fn_terse())

In [12]:
%timeit fn_expression()
%timeit fn_terse()

100 loops, best of 3: 7.61 ms per loop
100 loops, best of 3: 2.43 ms per loop


In [14]:
# dis查看两个函数的字节码指令行数
import dis
dis.dis(fn_expression)

  3           0 LOAD_CONST               1 (0)
              3 STORE_FAST               1 (total)

  4           6 SETUP_LOOP              30 (to 39)
              9 LOAD_GLOBAL              0 (range)
             12 LOAD_FAST                0 (upper)
             15 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             18 GET_ITER
        >>   19 FOR_ITER                16 (to 38)
             22 STORE_FAST               2 (n)

  5          25 LOAD_FAST                1 (total)
             28 LOAD_FAST                2 (n)
             31 INPLACE_ADD
             32 STORE_FAST               1 (total)
             35 JUMP_ABSOLUTE           19
        >>   38 POP_BLOCK

  6     >>   39 LOAD_FAST                1 (total)
             42 RETURN_VALUE


In [15]:
dis.dis(fn_terse)

  9           0 LOAD_GLOBAL              0 (sum)
              3 LOAD_GLOBAL              1 (range)
              6 LOAD_FAST                0 (upper)
              9 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             12 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             15 RETURN_VALUE


In [19]:
?sum()

In [ ]:
# chapter 3 列表和元组
### 3.1 更有效的搜索
#### 二叉树
def binary_search(needle, haystack):
      imin, imax = 0, len(haystack)
    while True:
        if imin >= imax:
            return -1
        midpoint = (imin + imax)//2
        if haystack[midpoint] > needle:
            imax = midpoint
        elif haystack[midpoint] < needle:
            imin = midpoint+1
        else:
            return midpoint

In [24]:
# 3-4 用bisect模块在列表中查询最接近目标的值
import bisect
import random
def find_closest(haystack, needle):
    # bisect_left will return the first value in the haystack
    i = bisect.bisect_left(haystack, needle)
    if i == len(haystack):
        return i - 1
    elif haystack[i] == needle:
        return i
    elif i>0:
        j = i - 1
        if haystack[i] - needle > needle - haystack[j]:
            return j 
    return i

importance_numbers = []
for i in range(10):
    new_number = random.randint(0, 1000)
    bisect.insort(importance_numbers, new_number)
print(importance_numbers)

closest_index = find_closest(importance_numbers, 500)
print(closest_index)
importance_numbers[closest_index]

[49, 75, 283, 351, 358, 497, 555, 610, 642, 756]
5


497

In [28]:
# 用列表和集合来查询不同的名字，主要是看复杂度
### 方法1：用列表的方式进行统计
def list_unique_names(phonebook):
    unique_names= []
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        for unique in unique_names:
            if unique == first_name:
                break
            else:
                unique_names.append(first_name)
    return len(unique_names)
               
### 方法2: 用集合的方式
def set_unique_names(phonebook):
    unique_names = set()
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        unique_names.add(first_name)
    return len(unique_names)

phonebook = [
    ("johe doe", "555-555-444454"),
    ("albert einstein", "212-555-444454"),
    ("jone murphey", "555-555-444454"),
    ("albert rutherford", "555-555-444454"),
    ("elaine bodian", "555-555-444454")
            ]
               
#列表的复杂度是O(nlog(n)),集合的方法是： O(n)             
%timeit set_unique_names(phonebook)
%timeit list_unique_names(phonebook)

The slowest run took 4.33 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.56 µs per loop
100000 loops, best of 3: 3.03 µs per loop


In [30]:
# 同一个类的两个实例一般是不同的，不会导致散列碰撞；
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

p1 = Point(1,1)
p2 = Point(1,1)
set([p1,p2])
Point(1,1) in set([p1,p2])

False

In [33]:
# 我们可以提供一个基于对象内容而不是对象的内存中的位置的自定义散列函数来解决这个问题；
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    def __hash__(self):
        """重新改写hash函数，用内容来进行hash"""
        return hash((self.x, self.y))
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

p1 = Point(1,1)
p2 = Point(1,1)
set([p1,p2])
Point(1,1) in set([p1,p2])    

True

In [36]:
# 最优双字母散列函数, 数值递增；
def twoletter_hash(key):
    offset = ord('a')
    k1, k2 = key
    return (ord(k2) - offset)+26*(ord(k1)-offset)

In [39]:
twoletter_hash('bb')

27

In [42]:
# 命名空间查询的降速效果；
# 解决方案是在循环开始之前设置一个本地变量保存一个函数全局引用；
from math import sin
def  tight_loop_slow(iterations):
    result = 0 
    for i in range(iterations):
        result += sin(i)

def tight_loop_fast(iterations):
    result = 0
    local_sin = sin
    for i in range(iterations):
        result += local_sin(i)
    
%timeit tight_loop_fast(10000000)
%timeit tight_loop_slow(10000000)

1 loop, best of 3: 1.97 s per loop
1 loop, best of 3: 2.2 s per loop


In [43]:
# 4.2 字典和命名空间解析
import math
from math import sin

def test1(x):
    return math.sin(x)

def test2(x):
    return sin(x)

def test3(x, sin=math.sin):
    return sin(x)

In [44]:
# 查看命名空间解析结果
dis.dis(test1)
dis.dis(test2)
dis.dis(test3)

  6           0 LOAD_GLOBAL              0 (math)
              3 LOAD_ATTR                1 (sin)
              6 LOAD_FAST                0 (x)
              9 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             12 RETURN_VALUE
  9           0 LOAD_GLOBAL              0 (sin)
              3 LOAD_FAST                0 (x)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 RETURN_VALUE
 12           0 LOAD_FAST                1 (sin)
              3 LOAD_FAST                0 (x)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 RETURN_VALUE


从结果可以看到，test1会经过两次字典查询，一次查找math模块，然后是查找sin函数;
test2从math中导入sin函数，在全局命名中直接访问；
test3以默认参数关键字的形式作为一个本地变量被保存在函数的定义中，无需字典查询，保存在一个微小的数组中，具有很快的查询速度。

In [47]:
# chapter 7: 异步编程介绍
from Queue import Queue
from functools import partial

eventloop = None
class EventLoop(Queue):
    def start(self):
        while True:
            function = self.get()
            function()
            
def do_hello():
    global eventloop
    print('hello')
    eventloop.put(do_world)

def do_world():
    global eventloop
    print('world')
    eventloop.put(do_hello)
    
if __name__ == "__main__":
    eventloop = EventLoop()
    eventloop.put(do_hello)
    eventloop.start()

ImportError: No module named 'Queue'